# Lumi FDTD with GPU acceleration & inverse design (experimental)
## Intro
Lumi FDTD by [Luminescent AI](https://luminescentai.com/) is a GPU and automatic differentiation (AD) compatible FDTD package for simulation and inverse design in semiconductor photonics. Create layout in gdsfactory. Then use the Lumi Python  plugin for calculating sparams or perform inverse design on Lumi's Julia backend. 

Experimental release . Expect bugs


## Gallery 
![](assets/demux.png)
Inverse Design of wavelength domain demultiplexer 
## Installation 
Install Python frontend . recommend Anaconda. Inside Anaconda Prompt do:
- `pip install luminescent `

Install Julia backend 
- Install latest stable  Julia from julialang.org
- Open Julia REPL via `julia` in terminal. Then install packages via  
- `]add UnPack, BSON,JSON,Dates, DataStructures,ImageTransformations, Meshes, CoordinateTransformations, GPUArraysCore, StatsBase, Zygote,Porcupine, Jello,ArrayPadding, AbbreviatedStackTraces,  Flux, FileIO,Images,CairoMakie,Functors,Lazy`

Optional
- If using GPU, install corresponding Julia GPU backend  
- `]add CUDA` or `AMDGPU` or `Metal`

## S-parameters calculation 
We support multi-wavelength, multi-mode, GPU-accelerated (optional ) sparams calculation . One can choose to do only a subset of sparams in case of symmetric devices by specifying `keys`, which can either be in short form eg "2,1" or long form eg `o2@0,o1@0` (`port@mode`). if `keys` omitted, all sparams will be calculated 

 Simulation or optimization results are saved to a timestamped folder inside `lumi_runs` in your pwd. Can load them later via `sol = load_solution(path_to_run)` (loads latest run if path omitted )

We demo on a tight waveguide bend

In [ ]:
import luminescent as lumi
from luminescent import LAYER, XMARGIN
import gdsfactory as gf
import pprint as pp

c = gf.components.bend_euler(radius=.75, allow_min_radius_violation=True)
c = lumi.add_bbox(c, layers=[LAYER.WGCLAD, LAYER.BOX], nonport_margin=XMARGIN)
c.show()

sol = lumi.write_sparams(c, wavelengths=[1.55], keys=["2,1"],
                         dx=0.05, approx_2D=False, gpu=None,)
# sol = lumi.load_solution()
pp.pprint(sol)


In [ ]:
{'sparams': {'1.55': {'o1@0,o1@0': (0.11776448-0.04094091j),
                      'o2@0,o1@0': (0.76620877-0.0066863224j)}},
 'tparams': {'1.55': {'o1@0,o1@0': 0.015544631, 'o2@0,o1@0': 0.5871206}}}

![](assets/bend.png)

You can see significant radiative loss from this very small radius bend

## Inverse design with GCells
GCells (generative cells) are a natural evolution of PCells (parametric cells) in semiconductor design . Given a set of inverse design objectives, a GCell will generate optimal geometry using adjoint optimization while ensuring manufacturability thru a minimal feature length `lmin`

We demo on inverse design of a wavelength domain demultiplexer splitting mixture of 850nm & 1550nm signals. We first load the `gcells.mimo`. It is just a gdsfactory component with static waveguide ports, a simple slab as pre-optimization geometry, and overlying rectangular design regions marked with the DESIGN layer.

In particular , MIMO GCell has m inputs on left and n outputs on right. We specify T-params target dictating all 1550nm signal to go from o1 to o2, and all 850nm signal to o3. By changing target , one can also design MMI splitters, mode converters , phase , filters in addition to wavelength domain multiplexers

  Adjoint optimization then optimizes geometry inside design regions with pre geometry as starting point . With `approx_2D=True`, we optimize in 2.5D which is significantly faster . However , for accuracy , the result must be finetuned in 3D, a feature that can be requested from Luminescent AI 

In [ ]:
from pprint import pprint
import gdsfactory as gf
import luminescent as lumi

name = "demux"
c = lumi.gcells.mimo(l=4.0, w=4.0, m=1, n=2, wwg=.5)
targets = {
    1.55: {
        "2,1": 1.0
    },
    .85: {
        "3,1": 1.0
    }}
c.show()

prob = lumi.inverse_design_problem(
    c, tparam_targets=targets,
    lmin=0.2, dx=0.05, maxiters=40, eta=10., approx_2D=True)
sol = lumi.solve(prob)

print("post optim tparams:")
pprint(sol["after"]["tparams"])

c = sol["after"]["component"]
c.write_gds(f"optimal_{name}.gds", "")
c.plot()


In [ ]:
post optim tparams:
{'0.85': {'o1@0,o1@0': 0.018236168,
          'o2@0,o1@0': 0.015066246,
          'o3@0,o1@0': 0.90159845},
 '1.55': {'o1@0,o1@0': 0.066036664,
          'o2@0,o1@0': 0.97269523,
          'o3@0,o1@0': 0.04481672}}

![](assets/demux.png)

AFter 40 iters loss continues decreasing and is within 10% of target . This is good spot to switch to 3D for finetuning and finishing the design 

## Geometry & materials 
Geometry is purely determined thru gdsfactory via component layout and layer stack. A gdsfactory component generates a 3D mesh which is clipped vertically some depths above and below the core layer . The material library maps the material tag of a layer in the layer stack to its property eg `MATERIAL_LIBRARY["Si"]["epsilon"]`

LAYER_STACK, LAYER, MATERIAL_LIBRARY corresponding to a SOI node are provided in `luminescent.generic_tech` similar to gdsfactory's. They can be customized and passed as `layer_stack`, `material_library` to `write_sparams, inverse_design_problem`

## GPU acceleration : not just CUDA
Use GPU by setting `gpu="CUDA"` (or AMDGPU, Metal). By default we use Nvidia CUDA but also support Intel OneAPI (pending), AMD ROCM & Apple Metal. However only CUDA has been extensively tested. Please contact us for support on non-CUDA alternative 

## Advanced features 
Following can be requested from info@luminescentai.com (mention your position and organization ):
- 3D inverse design 
- inverse design on GPU
- fp16 (vs fp32 by default )

## Community support 
File issues & discussions on [GitHub](https://github.com/paulxshen/Luminescent.jl). Please star us if you found us helpful :) Private messages to info@luminescentai.com or pxshen@alumni.stanford.edu . Stay on top of new features and bug fixes by signing up for our [newsletter](https://docs.google.com/forms/d/e/1FAIpQLSccBKPX13H1wlXjJutSuTtc2su6VvIhNYKBj0k3tij_oQJzDQ/viewform?pli=1) 